In [1]:
import  nltk.translate.bleu_score as bleu

In [2]:
reference_translation=['The cat is on the mat.'.split(),
                       'There is a cat on the mat.'.split()
                      ]
candidate_translation_1='the the the mat on the the.'.split()
candidate_translation_2='The cat is on the mat.'.split()

In [3]:
print("BLEU Score: ",bleu.sentence_bleu(reference_translation, candidate_translation_2))

BLEU Score:  1.0


In [4]:
print("BLEU Score: ",bleu.sentence_bleu(reference_translation, candidate_translation_1))

BLEU Score:  6.968148412761692e-155


C:\Users\khandelwalr\AppData\Local\Continuum\anaconda3\lib\site-packages\nltk\translate\bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\khandelwalr\AppData\Local\Continuum\anaconda3\lib\site-packages\nltk\translate\bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [5]:
import numpy as np
from nltk import ngrams
from collections import Counter

In [6]:
def count_ngram(unigram, ngram=1):
    """
    Return
    -----
    counter: dict, containing ngram as key, and count as value
    """
    return Counter(ngrams(unigram, ngram))

In [8]:
translation = 'It is a guide to action which ensures that the military always obeys the commands of the party.'.split()
list_of_references=[
    'It is a guide to action that ensures that the military will forever heed Party commands.',
    'It is the guiding principle which guarantees the military forces always being under the command of the Party.',
    'It is the practical guide for the army always to heed the directions of the party.'

]
res = count_ngram(translation)
print('unigram for translation {}'.format(res))
res = count_ngram(translation, 2)
print('Bigram for translation {}'.format(res))

unigram for translation Counter({('the',): 3, ('It',): 1, ('is',): 1, ('a',): 1, ('guide',): 1, ('to',): 1, ('action',): 1, ('which',): 1, ('ensures',): 1, ('that',): 1, ('military',): 1, ('always',): 1, ('obeys',): 1, ('commands',): 1, ('of',): 1, ('party.',): 1})
Bigram for translation Counter({('It', 'is'): 1, ('is', 'a'): 1, ('a', 'guide'): 1, ('guide', 'to'): 1, ('to', 'action'): 1, ('action', 'which'): 1, ('which', 'ensures'): 1, ('ensures', 'that'): 1, ('that', 'the'): 1, ('the', 'military'): 1, ('military', 'always'): 1, ('always', 'obeys'): 1, ('obeys', 'the'): 1, ('the', 'commands'): 1, ('commands', 'of'): 1, ('of', 'the'): 1, ('the', 'party.'): 1})


In [11]:
res = dict()
# retrieve translation unigram counts
ct_translation_u = count_ngram(translation, ngram=1)
ct_translation_u

Counter({('It',): 1,
         ('is',): 1,
         ('a',): 1,
         ('guide',): 1,
         ('to',): 1,
         ('action',): 1,
         ('which',): 1,
         ('ensures',): 1,
         ('that',): 1,
         ('the',): 3,
         ('military',): 1,
         ('always',): 1,
         ('obeys',): 1,
         ('commands',): 1,
         ('of',): 1,
         ('party.',): 1})

In [12]:
res = dict()
# retrieve translation unigram counts
ct_translation_b = count_ngram(translation, ngram=2)
ct_translation_b

Counter({('It', 'is'): 1,
         ('is', 'a'): 1,
         ('a', 'guide'): 1,
         ('guide', 'to'): 1,
         ('to', 'action'): 1,
         ('action', 'which'): 1,
         ('which', 'ensures'): 1,
         ('ensures', 'that'): 1,
         ('that', 'the'): 1,
         ('the', 'military'): 1,
         ('military', 'always'): 1,
         ('always', 'obeys'): 1,
         ('obeys', 'the'): 1,
         ('the', 'commands'): 1,
         ('commands', 'of'): 1,
         ('of', 'the'): 1,
         ('the', 'party.'): 1})

In [13]:
def count_clip_ngram(translation_u, list_of_reference_u, ngram=1):
    """
    Return
   clipped counts of the ngram for candidate and reference translation
    
    """
    res = dict()
    # retrieve hypothesis counts
    ct_translation_u = count_ngram(translation_u, ngram=ngram)
    
    # retrieve translation candidate counts
    for reference_u in list_of_reference_u:
        ct_reference_u = count_ngram(reference_u, ngram=ngram)
        for k in ct_reference_u:
            if k in res:
                res[k] = max(ct_reference_u[k], res[k])
            else:
                res[k] = ct_reference_u[k]
                

    return {
        k: min(ct_translation_u.get(k, 0), res.get(k, 0)) 
        for k in ct_translation_u
    }

In [15]:
res = count_clip_ngram(
    translation, 
    list(map(lambda ref: ref.split(), list_of_references))
)
print("Clipped counts for unigram between candidate and reference", res)

Clipped counts for unigram between candidate and reference {('It',): 1, ('is',): 1, ('a',): 1, ('guide',): 1, ('to',): 1, ('action',): 1, ('which',): 1, ('ensures',): 1, ('that',): 1, ('the',): 3, ('military',): 1, ('always',): 1, ('obeys',): 0, ('commands',): 0, ('of',): 1, ('party.',): 1}


In [16]:
def modified_precision(translation, list_of_references, ngram=1):
    """
    Return
    modified precision = clipped counts/ no. of unclipped candidate n-gram
    
    """
    ct_clip = count_clip_ngram(translation, list_of_references, ngram)
    ct = count_ngram(translation, ngram)
    
    return sum(ct_clip.values()) / float(max(sum(ct.values()), 1))

In [17]:
res_mp= modified_precision(translation, list_of_references)
print("Modofied precision", res_mp)

Modofied precision 0.05555555555555555


In [18]:
def closest_ref_length(translation_u, list_of_reference_u):
    """
    determine the closest reference length from translation length
    """
    len_trans = len(translation_u)
    closest_ref_idx = np.argmin([abs(len(x) - len_trans) for x in list_of_reference_u])
    return len(list_of_reference_u[closest_ref_idx])

In [19]:
ref_length = closest_ref_length(translation, list_of_references)
ref_length

82

In [20]:
def brevity_penalty(translation_u, list_of_reference_u):
    """
    """
    c = len(translation_u)
    r = closest_ref_length(translation_u, list_of_reference_u)
    
    if c > r:
        return 1
    else:
        return np.exp(1 - float(r)/c)

In [22]:
print("BP", brevity_penalty(translation, list_of_references))

BP 0.028565500784550377


In [23]:
def bleu_score(translation_u, list_of_reference_u, W=[0.25 for x in range(4)]):
    bp = brevity_penalty(translation_u, list_of_reference_u)
    modified_precisions = [
        modified_precision(translation_u, list_of_reference_u, ngram=ngram)
        for ngram, _ in enumerate(W,start=1)
    ]
    score = np.sum([
        wn * np.log(modified_precisions[i]) if modified_precisions[i] != 0 else 0 for i, wn in enumerate(W)
    ])
    
    return bp * np.exp(score)

In [24]:
print("BLEU", bleu_score(translation, list_of_references))

BLEU 0.013868315585598298
